# NEWS HEADLINE COMPILER 

![NEWS](http://fivebars.co.uk/wp-content/uploads/2018/08/news-1.jpg)

 - [Overview](#OVERVIEW)
 - [Instructions](#INSTRUCTIONS)
 - [Pip Requirements](#PIP-REQUIREMENTS)
 - [Selected Sources](#SELECTED-NEWS-SOURCES)
 - [Available Sources](#AVAILABLE-NEWS-SOURCES)

# OVERVIEW

This project utilises the awesome  [newsAPI](https://newsapi.org/s/uk-news-api) , to pull down the latest news articles, descriptions and other information from a select amount of news sources. This project then compiles those lists into a daily folder, as the API can only get a select slice of the news, it will run three times a day to build up news data which in the future could be used to drive text generation, news comparisons and so on.

Because the API returns json, it's easier to just save json files for each source, then collate those files into one CSV. This can then be appeneded throughout the day, week and year.  


This project is fully open source and mostly for fun, if any future work is aimed at generating revenue, you must comply with the newsAPI policy and upgrade from Developer. I may do this if I find a useful business case based upon my exploration here. 

This code requries access to newsAPI, thus can't run in this kernel: please reference my git repo for my latest updates [News_Source_compiler](https://github.com/murchie85/News_Source_compiler) . I will however include the code here anyway for completeness.
  

## INSTRUCTIONS

1. Sign up to [newsAPI](https://newsapi.org/s/uk-news-api), generate keys **(it will tell you how to)** and store in a parent directory of this one called `keys`, or simply change the key path in the python code. 
2. Create a folder called `data` at same level as the python code
3. run `python process_news.py` from command line / terminal.
4. Program will pull down news articles and store them in data folder, creating a new folder for each day you run it. 

  
## PIP REQUIREMENTS

```
pandas
json
os
datetime
newsapi
```


## SELECTED NEWS SOURCES 

I may change this depending on the circumstance.


```
news_keyname_array = ['bbc-news', 'abc-news','cnn','fox-news','independent','mirror','metro','daily-mail', 'Theguardian.com' , 'Sky.com', 'the-new-york-times', 'al-jazeera-english', 'reuters', 'the-hill' , 'breitbart-news', 'the-verge', 'the-huffington-post']
```

## AVAILABLE NEWS SOURCES

From calling `the get_sources.py` code i have written (because the website doesn't give a full list), the below list of available ones are there. May have different ones depending on time of day? 

```
abc-news-au
aftenposten
al-jazeera-english
ansa
argaam
ars-technica
ary-news
associated-press
australian-financial-review
axios
bbc-news
bbc-sport
bild
blasting-news-br
bleacher-report
bloomberg
breitbart-news
business-insider
business-insider-uk
buzzfeed
cbc-news
cbs-news
cnbc
cnn
cnn-es
crypto-coins-news
der-tagesspiegel
die-zeit
el-mundo
engadget
entertainment-weekly
espn
espn-cric-info
financial-post
focus
football-italia
fortune
four-four-two
fox-news
fox-sports
globo
google-news
google-news-ar
google-news-au
google-news-br
google-news-ca
google-news-fr
google-news-in
google-news-is
google-news-it
google-news-ru
google-news-sa
google-news-uk
goteborgs-posten
gruenderszene
hacker-news
handelsblatt
ign
il-sole-24-ore
independent
infobae
info-money
la-gaceta
la-nacion
la-repubblica
le-monde
lenta
lequipe
les-echos
liberation
marca
mashable
medical-news-today
msnbc
mtv-news
mtv-news-uk
national-geographic
national-review
nbc-news
news24
new-scientist
news-com-au
newsweek
new-york-magazine
next-big-future
nfl-news
nhl-news
nrk
politico
polygon
rbc
recode
reddit-r-all
reuters
rt
rte
rtl-nieuws
sabq
spiegel-online
svenska-dagbladet
t3n
talksport
techcrunch
techcrunch-cn
techradar
the-american-conservative
the-globe-and-mail
the-hill
the-hindu
the-huffington-post
the-irish-times
the-jerusalem-post
the-lad-bible
the-new-york-times
the-next-web
the-sport-bible
the-times-of-india
the-verge
the-wall-street-journal
the-washington-post
the-washington-times
time
usa-today
vice-news
wired
wired-de
wirtschafts-woche
xinhua-net
ynet
```

**Powered by news API**
[link](https://newsapi.org/s/uk-news-api)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
#-----------------------------------------------------------------------------------------
#
# Program name : process_news.py
# Program Created Date: 31 December 2019
# Program Amanded Date: 02 Jan 2020
# Program Author: Adam McMurchie
#
# Program Description:  This program iterrogates news.api, pulls data and consolodates in a re-occuring append fasion. The aim to 
#                       Overtime build a portfolio of news data from various sources. Various archival, and time delay considerations 
#                       have been baked into the code (explained in program flow).
#
# Input Files:     Data from news.api which is whatever was present 15 minutes ago. 
# Output Files:    Json files in data folder, output.csv full list in data folder
#
# Program Flow :  **to be updated** Run job three times per day (at least)
#                  Gets current date
#                  Checks current date data folder (create if not exist)
#                  compares news array pulled from news.api to written files
#                  If news array has items that are not in written files
#                  Regenerate news array with missing items
#                  call news api, get news for updated news array
#                  Save to json
#                  Also save all files to csv (can overwrite)
#                  Saves crossing any wires or putting the wrong list in the wrong market
#
#-----------------------------------------------------------------------------------------


# IMPORT STATEMENTS 
import json
import os
from datetime import date
from newsapi import NewsApiClient
import pandas as pd

# IMPORT API KEYS (YOU WILL NEED TO REGISTER AT NEWSAPI.ORG)
f = open("../your/key/location/api.txt", "r")
keys = f.read()
f.close()
ACCESS_TOKEN = keys



In [ ]:
#-------------------------------------------------------------------------------------------
#   PARMS SECTION 
#-------------------------------------------------------------------------------------------

# READING IN 
NewsFileNameArray = []                         # A LIST OF FILENAMES 
data = []                                      # ALL NEWS DATA FROM LOCAL NEWS JSON FILES

# REQUESTING NEW DATA
newsapi = NewsApiClient(api_key=ACCESS_TOKEN)  # INITIALISING newsapi object
news_keyname_array = ['bbc-news', 'abc-news','cnn','fox-news','independent','mirror','metro','daily-mail', 'Theguardian.com' , 'Sky.com', 'the-new-york-times', 'al-jazeera-english', 'reuters', 'the-hill' , 'breitbart-news', 'the-verge', 'the-huffington-post']
news_array = []



In [ ]:

#-------------------------------------------------------------------------------------------
#   CREATE NEW FOLDER FOR TODAY IF NOT EXIST
#-------------------------------------------------------------------------------------------
# GET TODAYS DATE 
today = date.today()
print("Today's date:", today)

# CREATE A NEW DIRECTORY FOR TODAY IF NOT EXIST
if os.path.isdir("data/" + str(today)):
    print('Dir Already exists')
else:
    os.mkdir("data/" + str(today))

print('')

In [ ]:
#-------------------------------------------------------------------------------------------
#   REQUESTING MORE NEWS 
#-------------------------------------------------------------------------------------------

# Init API and save to news_array
# WRITE TO JSON
print('PULLING NEWS HEADLINES - PLEASE WAIT .... ')
print('')
for item in news_keyname_array:
    print('processing ' + str(item + ' headlines'))
    news_key = item
    json_item = newsapi.get_top_headlines(sources=news_key)
    if json_item['totalResults'] == 0:
        print("Request for the " + str(item) + " news source is empty, skipping")
        print('')
        continue
    news_array.append(json_item)
    print('COMPLETE - appending to array .......')
    print('')


In [ ]:
#-------------------------------------------------------------------------------------------
#   BUILDING REPORT 
#-------------------------------------------------------------------------------------------

# BUILD A PANDAS DATA FRAME 
df = pd.DataFrame(columns=['source','author','title','description','url', 'requested_date','publishedAt','content'])


# Iterate through DATA array and write to csv
print('iterating through ')
x = 0 
for news_outlet in range (0, len(news_array)):
    for article_number in range (0, len(news_array[news_outlet]['articles'])):
        source         = news_array[news_outlet]['articles'][article_number]['source']['name']
        author         = news_array[news_outlet]['articles'][article_number]['author']
        title          = news_array[news_outlet]['articles'][article_number]['title']
        description    = news_array[news_outlet]['articles'][article_number]['description']
        url            = news_array[news_outlet]['articles'][article_number]['url']
        publishedAt    = news_array[news_outlet]['articles'][article_number]['publishedAt']
        requested_date = today
        content        = news_array[news_outlet]['articles'][article_number]['content']
        df = df.append([{ 'source': source, 'author': author, 'title': title, 'description': description, 'url':url, 'publishedAt': publishedAt, 'requested_date': requested_date, 'content': content    }])
        x = x + 1 

print('PROCESSING COMPLETE')
print('number of articles processed are : ' + str(x))

# imported is old data
# df is the new data
# combined is merged 


try:
    f = open("data/" + str(today) + "/" + 'output.csv')
    print('File exists')
except IOError:
    print("File not accessible, saving current df to file")
    df.to_csv("data/" + str(today) + "/" + 'output.csv')

finally:
    f.close()

print('importing saved data')
imported = pd.read_csv("data/" + str(today) + "/" + 'output.csv', index_col=0)
print('Appending old and new feeds')
combined = imported.append(df)
print('')
print('dropping duplicates')
combined = combined.drop_duplicates(subset="title", keep='first')
print('')
print('describing new data')
print(df.shape)
print('')
print('describing old data')
print(imported.shape)
print('describing merged data with dropped duplicates')
print(combined.shape)

        
combined.to_csv("data/" + str(today) + "/" + 'output.csv')